In [13]:
import importlib
import keys
importlib.reload(keys)
import json
import pprint
keychain = keys.keychain

pp = pprint.PrettyPrinter(indent=2)

client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
print(client_id[:20], scope, redirect)
keychain['youtube']

126920892433-e58m7pn ['https://www.googleapis.com/auth/youtube.readonly'] urn:ietf:wg:oauth:2.0:oob


{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [2]:
from requests_oauthlib import OAuth2Session

In [3]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
auth_url, state = session.authorization_url(keychain['youtube']['auth_uri'],access_type='offline',include_granted_scopes='true')
print(auth_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=0SycnEMT14qEX9RAMZ8ehTS1hAkFpH&access_type=offline&include_granted_scopes=true


In [6]:
code = '4/xe4WcQeCYyzXOUmfymrLdnPc6-JEZTNG_PC4x7_Ni8I'

In [7]:
token = session.fetch_token(keychain['youtube']['token_uri'],
                            code=code,
                            client_secret=keychain['youtube']['client_secret'])  

In [11]:
refresh_url = keychain['youtube']['token_uri']
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'])
token


{'access_token': 'ya29.GlscBZER4fe-lFL7S1aX8vvTttD78-vuqhcrSH_MJWIf7aDMvZ88VX8DVG8gDhr42KVrjbB2BMFjt70nNMWfHiDvJLQVEN8P2xQzAlLC5tPanCxijkvI1H6MN68f',
 'expires_at': 1512716300.6317785,
 'expires_in': 3600,
 'refresh_token': '1/404Y5NzJtD0RmPCDMHRJ2QOHgM1dHW3fkogwXyJZiMQ',
 'token_type': 'Bearer'}

In [9]:
import json
with open('token.json', 'w') as outfile:
    json.dump(token, outfile)